In [ ]:
import os
import logging
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Configure logging for detailed debug information
logging.basicConfig(level=logging.DEBUG)

# Load OpenAI API key from environment variable
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API key not found. Please set it in your .env file.")

# OpenAI's LLM initialization
llm = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo", api_key=api_key)


# Define the schema for the tool's arguments
class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for. Example query: Stock Market Symbol for Apple Company"
    )


# Define the stock market symbol search tool
class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = (
        StockMarketSymbolSearchToolArgsSchema
    )

    def _run(self, query: str):
        ddg = DuckDuckGoSearchAPIWrapper()
        try:
            logging.debug(f"Running DuckDuckGo search with query: {query}")
            results = ddg.results(query)
            logging.debug(f"DuckDuckGo search results: {results}")
            if not results:
                return "No results found."
            return results
        except Exception as e:
            logging.error(f"Error during DuckDuckGo search: {e}")
            return f"Error during DuckDuckGo search: {e}"


# Create a structured prompt template
prompt_template = PromptTemplate(
    input_variables=["query", "tools", "tool_names", "agent_scratchpad"],
    template="""
    You are an AI assistant. Use the provided tools to help the user with their queries.
    The user will ask for information about various companies and their stock market symbols.
    The user's query is: {query}

    Tools available: {tool_names}
    {tools}
    
    {agent_scratchpad}
    """,
)

# Initialize the structured chat agent with the prompt template
agent = create_structured_chat_agent(
    llm=llm,
    tools=[StockMarketSymbolSearchTool()],
    prompt=prompt_template,
)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent,
    tools=[StockMarketSymbolSearchTool()],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1,  # Limit the number of iterations to avoid infinite loops
)

# Define the input prompt
input_prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

# Run the agent with the input prompt
try:
    logging.debug(f"Invoking agent with input prompt: {input_prompt}")
    response = agent_executor.invoke({"query": input_prompt})
    logging.debug(f"Agent response: {response}")
    print(response)
except IndexError as e:
    logging.error(f"IndexError during agent execution: {e}")
    print(f"IndexError during agent execution: {e}")
except Exception as e:
    logging.error(f"Error during agent execution: {e}")
    print(f"Error during agent execution: {e}")

In [ ]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(temperature=0.1)

db = SQLDatabase.from_uri("sqlite:///moviews.sqlite")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)